In [10]:
import pandas as pd
import pandas_datareader.data as pdr
from pandas import Series, DataFrame
import numpy as np

import datetime
from datetime import date, timedelta
import yfinance as yf
yf.pdr_override()

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def fullPrint(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)   


In [11]:
df = pdr.get_data_yahoo("vrtx", start="2016-01-01",end="2021-11-26")
sp = "^GSPC"
spdf = pdr.get_data_yahoo(sp, start="2016-01-01",end="2021-11-26")
sortLogic = {
    'Close': 'last',
    'Volume': 'sum'
}
dfSorted = df.resample("W-FRI").agg(sortLogic)
spDfSorted = spdf.resample("W-FRI").agg(sortLogic)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [12]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
dfSorted['30WMA'] = dfSorted['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)

In [13]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=dfSorted.index, y=dfSorted['Close'],name='Price'),secondary_y=True)
fig.add_trace(go.Scatter(x=dfSorted.index, y=dfSorted['30WMA'],name="30WMA"),secondary_y=True)
fig.add_trace(go.Bar(x=dfSorted.index, y=dfSorted['Volume'],name="Volume"),secondary_y=False)
fig.show()

In [14]:
deltaX = 10.4
dfSorted['30WMASlope'] = dfSorted['30WMA'].diff()*500/(dfSorted['Close'].rolling(window=104).max()-dfSorted['Close'].rolling(window=104).min())/deltaX

In [15]:
dfSorted['P4WH'] = dfSorted['Close'].rolling(window=4).max()
dfSorted['P4WL'] = dfSorted['Close'].rolling(window=4).min()


In [16]:
dfSorted['Percent'] = dfSorted['Close'].pct_change()
dfSorted['VolumePerc'] = dfSorted['Volume'].pct_change() 
spDfSorted['Percent'] = spDfSorted['Close'].pct_change()
dfSorted['RS'] = dfSorted['Percent'] - spDfSorted['Percent']
dfSorted

,Close,Volume,30WMA,30WMASlope,P4WH,P4WL,Percent,VolumePerc,RS
Date,,,,,,,,,
2016-01-08,110.709999,9395500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-15,97.910004,19104200,NaN,NaN,NaN,NaN,-0.115617,1.033335,-0.093921
2016-01-22,94.709999,12322600,NaN,NaN,NaN,NaN,-0.032683,-0.354980,-0.046814
2016-01-29,90.750000,20427300,NaN,NaN,110.709999,90.750000,-0.041812,0.657710,-0.059296
2016-02-05,86.610001,16337100,NaN,NaN,97.910004,86.610001,-0.045620,-0.200232,-0.014598
...,...,...,...,...,...,...,...,...,...
2021-10-29,184.929993,6127700,192.884926,-0.402733,184.929993,178.940002,0.007683,-0.358013,-0.005624
2021-11-05,192.850006,13108400,192.488066,-0.165064,192.850006,181.770004,0.042827,1.139204,0.022818
2021-11-12,187.289993,6499800,191.776582,-0.295925,192.850006,183.520004,-0.028831,-0.504150,-0.025706


In [23]:
def checkIfStage1(price,volumePerc, RS, slope, WMA,P4WH,P4WL):
    stageOneIndicator = 0
    falseReason = ""
    if volumePerc<0.2:
        stageOneIndicator+=1
    else:
        falseReason += "volume "
    if slope<=0.8 and slope>=-0.3:
        stageOneIndicator+=1
    else:
        falseReason += "slope "
    if P4WH<P4WL*1.15:
        stageOneIndicator+=1
    else:
        falseReason += "range "
    if price <= WMA*1.1 and price>= WMA*0.9:
        stageOneIndicator+=1
    else:
        falseReason += "price "
    if stageOneIndicator>=3:
        return True
    return "False " + falseReason

In [24]:
dfSorted['IsStage1'] = dfSorted.apply(lambda x: checkIfStage1(x['Close'],x['VolumePerc'],x['RS'],x['30WMASlope'],x['30WMA'],x['P4WH'],x['P4WL']),axis=1)
fullPrint(dfSorted)

                 Close    Volume       30WMA  30WMASlope        P4WH  \
Date                                                                   
2016-01-08  110.709999   9395500         NaN         NaN         NaN   
2016-01-15   97.910004  19104200         NaN         NaN         NaN   
2016-01-22   94.709999  12322600         NaN         NaN         NaN   
2016-01-29   90.750000  20427300         NaN         NaN  110.709999   
2016-02-05   86.610001  16337100         NaN         NaN   97.910004   
2016-02-12   83.230003  19156200         NaN         NaN   94.709999   
2016-02-19   88.080002   8631000         NaN         NaN   90.750000   
2016-02-26   87.000000   7963000         NaN         NaN   88.080002   
2016-03-04   88.360001   9806800         NaN         NaN   88.360001   
2016-03-11   86.260002  10793700         NaN         NaN   88.360001   
2016-03-18   82.519997  13160500         NaN         NaN   88.360001   
2016-03-25   80.580002   7655600         NaN         NaN   88.36